<a href="https://colab.research.google.com/github/King-Rian/Project-3/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from google.colab import files
uploaded = files.upload()


Saving emojis.json.zip to emojis.json (2).zip


In [59]:
from google.colab import files
import zipfile
import json
import pandas as pd

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the ZIP file
with zipfile.ZipFile('emojis.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Load the extracted JSON file
emoji_file = 'emojis.json'

# Using pandas
emoji_df = pd.read_json(emoji_file)
print("Dataset loaded successfully. Sample data:")
print(emoji_df.head())


Saving emojis.json.zip to emojis.json (4).zip
Dataset loaded successfully. Sample data:
                                            category  \
0  Miscellaneous Symbols And Pictographs -> Emoji...   
1  Miscellaneous Symbols And Pictographs -> Emoji...   
2                                               None   
3  Miscellaneous Symbols And Pictographs -> Emoji...   
4                                               None   

                                            keywords  \
0  [dark skin tone, hand, forbidden, gesture, wom...   
1                     [dark skin tone, woman, guard]   
2                 [racing, running, woman, marathon]   
3  [gymnastics, medium-light skin tone, woman, ca...   
4                                      [woman, golf]   

                                          definition  \
0  The Woman Gesturing Not OK, Type-6 emoji is a ...   
1  The Female Guard, Type-6 emoji is a sequence o...   
2  The female version of the ?? Runner emoji. The...   
3  The Woman D

# Clean the data

In [60]:
# Check data types and column names
emoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    1988 non-null   object
 1   keywords    2389 non-null   object
 2   definition  2389 non-null   object
 3   unicode     2389 non-null   object
 4   name        2389 non-null   object
 5   shortcode   845 non-null    object
 6   senses      2389 non-null   object
dtypes: object(7)
memory usage: 130.8+ KB


In [61]:
# Check for null values
emoji_df.isnull().sum()


,0
category,401
keywords,0
definition,0
unicode,0
name,0
shortcode,1544
senses,0


In [62]:
# Show unique values for shortcode
unique_values = emoji_df['shortcode'].unique()
print(unique_values)

[None ':zero:' ':m:' ':accept:' ':ideograph_advantage:' ':u55b6:'
 ':u5272:' ':u7533:' ':u6708:' ':u6709:' ':u6e80:' ':u5408:' ':u7a7a:'
 ':u7981:' ':u6307:' ':u7121:' ':sa:' ':koko:' ':kr:' ':cn:'
 ':hourglass_flowing_sand:' ':alarm_clock:' ':arrow_double_down:'
 ':arrow_double_up:' ':rewind:' ':fast_forward:' ':eight:' ':de:' ':four:'
 ':vs:' ':up:' ':sos:' ':ok:' ':ng:' ':new:' ':id:' ':free:' ':cool:'
 ':cl:' ':ab:' ':parking:' ':o2:' ':b:' ':a:' ':hourglass:' ':watch:'
 ':ru:' ':black_joker:' ':seven:' ':three:' ':arrow_heading_down:'
 ':arrow_heading_up:' ':registered:' ':copyright:' ':left_luggage:'
 ':baggage_claim:' ':customs:' ':passport_control:' ':bathtub:' ':bath:'
 ':shower:' ':wc:' ':toilet:' ':baby_symbol:' ':restroom:' ':womens:'
 ':mens:' ':children_crossing:' ':no_pedestrians:' ':walking:'
 ':mountain_bicyclist:' ':bicyclist:' ':no_bicycles:' ':bike:'
 ':non-potable_water:' ':potable_water:' ':do_not_litter:'
 ':put_litter_in_its_place:' ':no_smoking:' ':smoking:' ':

In [63]:
# Show unique values for category
unique_values = emoji_df['category'].unique()
print(unique_values)

['Miscellaneous Symbols And Pictographs -> Emoji modifiers' None
 'Miscellaneous Symbols And Pictographs -> Hand symbols'
 'Supplemental Symbols And Pictographs -> Animal symbols'
 'Supplemental Symbols And Pictographs -> Food symbol'
 'Supplemental Symbols And Pictographs -> Miscellaneous mark'
 'Miscellaneous Symbols -> Map symbol from ARIB STD B24'
 'Supplemental Symbols And Pictographs -> Portrait and role symbols'
 'Supplemental Symbols And Pictographs -> Emoticon faces'
 'Supplemental Symbols And Pictographs -> Hand symbols'
 'Miscellaneous Symbols And Pictographs -> Sport symbols'
 'Enclosed Alphanumerics -> Circled Latin letters'
 'Miscellaneous Symbols And Pictographs -> Portrait and role symbols'
 'Enclosed Ideographic Supplement -> Circled ideographs'
 'Enclosed Ideographic Supplement -> Squared ideographs'
 'Enclosed Ideographic Supplement -> Squared ideographs and kana from ARIB STD B24'
 'Enclosed Ideographic Supplement -> Squared katakana'
 'Miscellaneous Technical -> Us

In [64]:
# Drop category and shortcode columns to improve model processing
emoji_df = emoji_df.drop(columns=['category', 'shortcode'])

In [65]:
emoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   keywords    2389 non-null   object
 1   definition  2389 non-null   object
 2   unicode     2389 non-null   object
 3   name        2389 non-null   object
 4   senses      2389 non-null   object
dtypes: object(5)
memory usage: 93.4+ KB


# Preprocess the data


In [66]:
# Step 4: Preprocess the data to map context words to emojis
def map_phrases_to_emojis(df):
    mapping = {}
    for _, row in df.iterrows():
        for word in row.get("context_words", []):  # Safely access context_words
            mapping[word] = row["emoji"]
    return mapping

emoji_mapping = map_phrases_to_emojis(emoji_df)
print("Sample emoji mapping:", list(emoji_mapping.items())[:5])


Sample emoji mapping: []


In [67]:
import nltk

# Redownload 'punkt' tokenizer
nltk.download('punkt', force=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [68]:
import nltk
import os

# Manually set the NLTK data directory
nltk_data_path = '/root/nltk_data'
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.append(nltk_data_path)

# Download the 'punkt' resource
nltk.download('punkt', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
import nltk
import os

# Download 'punkt_tab'
nltk.download('punkt_tab')

# Ensure NLTK data path is set correctly (if needed)
nltk_data_path = '/root/nltk_data'
os.makedirs(nltk_data_path, exist_ok=True)  # Create if doesn't exist
nltk.data.path.append(nltk_data_path)      # Add to NLTK's search path

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [70]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Step 5: Tokenize and map text to emoji sequences
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    return [emoji_mapping.get(token, token) for token in tokens]

# Example text preprocessing
text = "I love sunny weather"
emoji_sequence = tokenize_text(text)
print("Input Text:", text)
print("Emoji Sequence:", emoji_sequence)


Input Text: I love sunny weather
Emoji Sequence: ['i', 'love', 'sunny', 'weather']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load a pre-trained transformer model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Encode input text
input_text = "I love sunny weather"
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate emoji sequence
outputs = model.generate(inputs["input_ids"], max_length=10, num_return_sequences=1)
emoji_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Emoji Sequence from Model:", emoji_sequence)


Generated Emoji Sequence from Model: i love love love love love love love love


In [72]:
!pip install vaderSentiment

In [74]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Refine emoji selection based on sentiment
def refine_with_sentiment(text, emoji_sequence):
    sentiment = analyzer.polarity_scores(text)
    tone = "positive" if sentiment["compound"] > 0 else "negative"
    refined_sequence = [emoji + ("😊" if tone == "positive" else "😢") for emoji in emoji_sequence]
    return refined_sequence

# Split the emoji sequence into a list of individual emojis
emoji_sequence_list = list(emoji_sequence)

refined_sequence = refine_with_sentiment(input_text, emoji_sequence_list)
print("Refined Emoji Sequence Based on Sentiment:", refined_sequence)

Refined Emoji Sequence Based on Sentiment: ['i😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊']


In [75]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Refine emoji selection based on sentiment
def refine_with_sentiment(text, emoji_sequence):
    sentiment = analyzer.polarity_scores(text)
    tone = "positive" if sentiment["compound"] > 0 else "negative"
    refined_sequence = [emoji + ("😊" if tone == "positive" else "😢") for emoji in emoji_sequence]
    return refined_sequence

refined_sequence = refine_with_sentiment(input_text, emoji_sequence.split())
print("Refined Emoji Sequence Based on Sentiment:", refined_sequence)


Refined Emoji Sequence Based on Sentiment: ['i😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊']


In [80]:
from transformers import AdamW, T5Tokenizer, AutoModelForSeq2SeqLM

# Load a pre-trained T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Use T5 tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Simple training loop (illustration, adjust as needed for your dataset)
for epoch in range(10):  # Iterate through epochs
    model.train()
    inputs = tokenizer("I love sunny weather", return_tensors="pt", max_length=512, truncation=True)
    labels = tokenizer("😊☀️", return_tensors="pt", max_length=10, truncation=True)["input_ids"]

    # Remove token_type_ids from inputs if present
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch + 1} Loss: {loss.item()}")

Epoch 1 Loss: 4.525148868560791
Epoch 2 Loss: 2.609675168991089
Epoch 3 Loss: 3.2535552978515625
Epoch 4 Loss: 4.122931003570557
Epoch 5 Loss: 3.5338964462280273
Epoch 6 Loss: 2.5128514766693115
Epoch 7 Loss: 3.537613868713379
Epoch 8 Loss: 1.5669277906417847
Epoch 9 Loss: 2.106816053390503
Epoch 10 Loss: 1.6705074310302734


In [81]:
from transformers import AdamW

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Simple training loop (illustration, adjust as needed for your dataset)
for epoch in range(10):  # Iterate through epochs
    model.train()
    inputs = tokenizer("I love sunny weather", return_tensors="pt", max_length=512, truncation=True)
    labels = tokenizer("😊☀️", return_tensors="pt", max_length=10, truncation=True)["input_ids"]

    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch + 1} Loss: {loss.item()}")


Epoch 1 Loss: 1.1596747636795044
Epoch 2 Loss: 0.7002083659172058
Epoch 3 Loss: 1.0428745746612549
Epoch 4 Loss: 0.8993375897407532
Epoch 5 Loss: 0.7519550323486328
Epoch 6 Loss: 0.31472471356391907
Epoch 7 Loss: 1.7767057418823242
Epoch 8 Loss: 1.0772424936294556
Epoch 9 Loss: 0.5401368141174316
Epoch 10 Loss: 0.4444301128387451


In [78]:
!pip install python-Levenshtein

In [79]:
from Levenshtein import distance as levenshtein_distance

# Evaluate emoji sequence matching
def evaluate_sequences(predicted, actual):
    score = levenshtein_distance(predicted, actual)
    return score

predicted_sequence = "😊☀️"
actual_sequence = "😊☀️"
score = evaluate_sequences(predicted_sequence, actual_sequence)
print("Levenshtein Distance Between Sequences:", score)

Levenshtein Distance Between Sequences: 0
